<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-14 09:28:36
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.36 C
-------------------
Today PnL: -65.35 K (-0.48%)
Current PnL: -14.55 L (-10.14%)
CY Booked + Current PnL: -7.59 L (-5.29%)
-------------------
Total profit:  4.51 L
Total loss:  -19.06 L
-------------------
Total Booked + Current PnL: 19.39 L (16.64%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.13%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 73.08 L (53.91%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
92      VIPIND   488.80   489.00  -217.75       H-SC  68.75   84922.0   
87  UJJIVANSFB    52.77    53.00    49.36       M-SC  44.79  129600.0   
71        SBIN   760.30   863.00   -11.51       M-LC  66.67  214425.0   
39    HINDALCO   651.95   761.55   -22.38       H-LC  71.88  107488.0   
4   APOLLOHOSP  6904.43  8285.00   -20.73       H-LC  62.50  108000.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
92     -10394.0  10428.0       -4.61        -10.90  12.28   0.04    157.0   
87     -12879.0  13504.0        0.97         -9.04  10.42   0.44    247.0   
71      12946.0  14281.0        0.06          6.43   6.66  13.51     77.0   
39       3176.0  14360.0        0.13          3.04  13.36  16.81     27.0   
4        4434.0  16276.0        0.15          4.28  15.07  20.00     32.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
92    -1.00        0.63    67.50    OX40N      NTT        MISC  
87    -0.95        0.96    52.96    OX40N      NTT       BANKS  
71     0.91        1.58    19.83     XY25      NTT       BANKS  
39     0.22        0.79    19.58      X5K      ATH      METALS  
4      0.27        0.80    18.96     X40N      BTT  HEALTHCARE

In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
14  BERGEPAINT   561.33  680.00   -19.06       H-MC  45.83  228967.0   
60  NATIONALUM   189.63  247.44   -50.15       H-MC  56.25   99900.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
14       1628.0  46435.0       -1.26          0.72  20.28  21.14    117.0   
60       -604.0  31239.0       -0.53         -0.60  31.27  30.49     97.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
14     0.04        1.69    28.20     XY24      NTT   PAINTS  
60    -0.02        0.74    31.61       MH      ATH   METALS

In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
90  VALIANTORG   512.64   838.00  -355.15       H-SC   4.17  108420.0   
41    HINDZINC   514.80   730.22    14.49       H-LC   1.04   86180.0   
88   UNIONBANK   123.87   163.00    -4.47       M-LC  46.88  165740.0   
45   INDIAMART  2327.09  4911.36   -46.21       H-SC  85.42  139888.0   
23   COALINDIA   406.65   512.15     1.52       L-LC  30.21  142413.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
90     -24866.0  109461.0        2.86        -18.66  100.96   63.47    151.0   
41     -16780.0   59861.0        1.41        -16.30   69.46   41.85     24.0   
88      24900.0   19590.0        1.41         17.68   11.82   31.59     88.0   
45      16552.0  120416.0        1.34         13.42   86.08  111.05    123.0   
23      -8048.0   47082.0        1.04         -5.35   33.06   25.94    162.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
90    -0.23        0.80    81.74       XR      NTT  CHEMICALS  
41    -0.28        0.64    13.42      X5K      ATH     METALS  
88     1.27        1.22    43.97     XY24      NTT      BANKS  
45     0.14        1.03    37.07       AR      ATH       MISC  
23    -0.17        1.05     8.67     XY25      ATH     MINING

In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  Current P/L  \
92  VIPIND   488.80   489.0  -217.75       H-SC  68.75   84922.0     -10394.0   
83     TCS  3794.03  4998.0   -21.92       X-LC   6.25  254748.0     -41186.0   
32   ENRIN  1377.95  3176.7      NaN        NaN  97.92   77525.0      43076.0   
63   QUESS   296.83   986.0   -32.78       M-SC  29.17   66543.0       1537.0   
30   DMART  3764.92  5201.0   -22.39       X-LC  26.04  194912.0      10431.0   

     FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  RRR Ind  \
92   10428.0       -4.61        -10.90   12.28    0.04    157.0    -1.00   
83  135093.0       -3.43        -13.92   53.03   31.73      1.0    -0.30   
32    1892.0       -2.38        125.04    2.44  130.54    274.0    22.77   
63  149389.0       -2.20          2.36  224.50  232.18    193.0     0.01   
30   59935.0       -2.13          5.65   30.75   38.14     15.0     0.17   

    CurrAlloc%  Gained% Criteria Strategy    Category  
92        0.63    67.50    OX40N      NTT        MISC  
83        1.88     1.97      X40      BTT          IT  
32        0.57    18.55       AR      ATH  ELECTRICAL  
63        0.49     1.89     XY24      NTT        MISC  
30        1.44    16.86     X40N      NTT        FMCG

In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
87  UJJIVANSFB    52.77    53.0    49.36       M-SC  44.79  129600.0   
92      VIPIND   488.80   489.0  -217.75       H-SC  68.75   84922.0   
79    SYMPHONY  1306.42  1306.0   -29.94       M-SC  51.04  149458.0   
46  INDIGOPNTS  1407.73  1408.0   126.56       H-SC  94.79  148688.0   
81   TATAELXSI  7332.28  7332.0   -18.82       X-MC  12.50   85218.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%  OTT%  CumlRnk  \
87     -12879.0  13504.0        0.97         -9.04  10.42  0.44    247.0   
92     -10394.0  10428.0       -4.61        -10.90  12.28  0.04    157.0   
79     -21683.0  21627.0       -0.73        -12.67  14.47 -0.03    191.0   
46     -25871.0  25901.0       -0.09        -14.82  17.42  0.02    133.0   
81     -17434.0  17427.0        0.35        -16.98  20.45 -0.00     59.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
87    -0.95        0.96    52.96    OX40N      NTT     BANKS  
92    -1.00        0.63    67.50    OX40N      NTT      MISC  
79    -1.00        1.10     7.42    OX40N      NTT  DURABLES  
46    -1.00        1.10    28.94    OX40N      NTT    PAINTS  
81    -1.00        0.63    29.54    OX40N      NTT        IT

In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
83         TCS  3794.03  4998.00   -21.92       X-LC   6.25  254748.0   
48        INFY  1461.46  2275.00   -12.06       X-LC  23.96  281639.0   
40  HINDUNILVR  2413.81  2723.00   -10.55       X-LC  90.62  280258.0   
17   BRITANNIA  4983.33  6446.05    15.59       X-LC  73.96  265144.0   
74     SIEMENS  4406.90  7969.85    -6.97       X-LC  18.75   78605.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
83     -41186.0  135093.0       -3.43        -13.92   53.03  31.73      1.0   
48      20038.0  125583.0       -1.35          7.66   44.59  55.67      2.0   
40       9911.0   24719.0       -0.69          3.67    8.82  12.81      5.0   
17      35911.0   31367.0        0.10         15.67   11.83  29.35     10.0   
74     -31567.0  120643.0        0.85        -28.65  153.48  80.85     11.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
83    -0.30        1.88     1.97      X40      BTT          IT  
48     0.16        2.08    14.23      X40      BTT          IT  
40     0.40        2.07    17.21     XY25      NTT        FMCG  
17     1.14        1.96    25.98     XY25      ATH        FMCG  
74    -0.26        0.58    15.77       AR      ATH  ELECTRICAL

In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction     RSP   Current  \
32    ENRIN  1377.95  3176.7      NaN        NaN   97.92   77525.0   
52  JPPOWER    18.73    26.2   -11.70       L-SC  100.00  177284.0   
59  MEDANTA  1064.65  1486.0    18.07       X-SC   87.50  251884.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
32      43076.0   1892.0       -2.38        125.04   2.44  130.54    274.0   
52      35442.0  21132.0       -0.93         24.99  11.92   39.88    262.0   
59      46407.0  34911.0        0.14         22.58  13.86   39.58     65.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
32    22.77        0.57    18.55       AR      ATH  ELECTRICAL  
52     1.68        1.31    79.66     XY24      NTT       POWER  
59     1.33        1.86    34.98     XY24      NTT  HEALTHCARE

In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
68        SAIL   130.64   228.00    91.93       M-MC  75.00  230249.0   
54    KPIGREEN   497.21   731.82    28.30       H-SC  80.21  131317.0   
20        CAMS  3643.00  5250.99     1.87       H-SC  48.96  114540.0   
15  BLUESTARCO  1646.70  2337.55    -4.20       H-SC  88.54  179750.0   
88   UNIONBANK   123.87   163.00    -4.47       M-LC  46.88  165740.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
68       5287.0  162372.0       -0.46          2.35  70.52  74.53    189.0   
54       6020.0   53105.0       -0.03          4.80  40.44  47.19    143.0   
20      12536.0   32484.0        0.52         12.29  28.36  44.14    126.0   
15      15080.0   53997.0       -0.13          9.16  30.04  41.95    124.0   
88      24900.0   19590.0        1.41         17.68  11.82  31.59     88.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
68     0.03        1.70    33.68     XY24      BTT    STEEL  
54     0.11        0.97    64.29       MH      ATH    POWER  
20     0.39        0.85    30.53     X40N      ATH     MISC  
15     0.28        1.33    17.31     X40N      ATH       AC  
88     1.27        1.22    43.97     XY24      NTT    BANKS

In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
32      ENRIN  1377.95  3176.70      NaN        NaN  97.92   77525.0   
94  WHIRLPOOL  1167.49  2270.00   -38.18       M-SC  79.17  203860.0   
45  INDIAMART  2327.09  4911.36   -46.21       H-SC  85.42  139888.0   
54   KPIGREEN   497.21   731.82    28.30       H-SC  80.21  131317.0   
95      WIPRO   243.46   420.00    -7.40       M-LC  27.08  158813.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
32      43076.0    1892.0       -2.38        125.04   2.44  130.54    274.0   
94      32239.0  129838.0        1.04         18.78  63.69   94.43    201.0   
45      16552.0  120416.0        1.34         13.42  86.08  111.05    123.0   
54       6020.0   53105.0       -0.03          4.80  40.44   47.19    143.0   
95       7868.0  101593.0       -0.83          5.21  63.97   72.51     71.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
32    22.77        0.57    18.55       AR      ATH  ELECTRICAL  
94     0.25        1.50    50.46       XR      NTT    DURABLES  
45     0.14        1.03    37.07       AR      ATH        MISC  
54     0.11        0.97    64.29       MH      ATH       POWER  
95     0.08        1.17     9.40       XR      NTT          IT

In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP  Current  \
24   COFFEEDAY    59.14    80.00   -55.73       L-SC  32.29  65779.0   
32       ENRIN  1377.95  3176.70      NaN        NaN  97.92  77525.0   
92      VIPIND   488.80   489.00  -217.75       H-SC  68.75  84922.0   
58      MASFIN   326.60   399.50   -19.60       H-SC  82.29  91620.0   
3   ALKYLAMINE  2347.98  4546.37     2.89       X-SC  95.83  95787.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
24     -47770.0  87822.0       -1.52        -42.07  133.51   35.27    265.0   
32      43076.0   1892.0       -2.38        125.04    2.44  130.54    274.0   
92     -10394.0  10428.0       -4.61        -10.90   12.28    0.04    157.0   
58      -6360.0  28228.0       -1.52         -6.49   30.81   22.32    145.0   
3       -5176.0  99705.0       -0.30         -5.13  104.09   93.63     67.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
24    -0.54        0.49    61.00       XR      NTT      HOTELS  
32    22.77        0.57    18.55       AR      ATH  ELECTRICAL  
92    -1.00        0.63    67.50    OX40N      NTT        MISC  
58    -0.23        0.68    32.88       XR      ATH     FINANCE  
3     -0.05        0.71    42.67       SR      ATH   CHEMICALS

In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
52     JPPOWER    18.73    26.20   -11.70       L-SC  100.00  177284.0   
65      RELAXO   902.64  1176.00   -30.57       H-SC   98.96   62784.0   
32       ENRIN  1377.95  3176.70      NaN        NaN   97.92   77525.0   
26       DABUR   505.20   735.00    -4.49       X-MC   96.88  206954.0   
3   ALKYLAMINE  2347.98  4546.37     2.89       X-SC   95.83   95787.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
52      35442.0  21132.0       -0.93         24.99   11.92   39.88    262.0   
65     -52754.0  87747.0       -2.06        -45.66  139.76   30.28    131.0   
32      43076.0   1892.0       -2.38        125.04    2.44  130.54    274.0   
26       9926.0  79698.0       -0.04          5.04   38.51   45.49     56.0   
3       -5176.0  99705.0       -0.30         -5.13  104.09   93.63     67.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
52     1.68        1.31    79.66     XY24      NTT       POWER  
65    -0.60        0.46    22.66     X40N      NTT    FOOTWEAR  
32    22.77        0.57    18.55       AR      ATH  ELECTRICAL  
26     0.12        1.53    17.56     XY24      BTT        FMCG  
3     -0.05        0.71    42.67       SR      ATH   CHEMICALS

In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.00
1     25       41.98
2     50       70.69

In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.72
LC    33.60
MC    21.65
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.14
X40      12.80
XY25     12.16
X40N     11.86
XR        9.97
OX40N     8.21
AR        8.19
MH        1.71
X5K       1.43
X200      1.36
SR        1.19
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.54
M-SC    15.91
H-LC    13.96
X-LC    12.80
H-MC     9.59
X-MC     7.63
M-LC     5.79
X-SC     4.57
M-MC     4.07
L-SC     2.70
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                15.32         -0.90   34.62
IT                  11.86        -13.12   68.72
FINANCE              8.46        -13.82   55.11
BANKS                7.70         -9.21   53.77
MISC                 5.88        -20.80   77.47
PAINTS               5.64        -14.42   36.97
ELECTRICAL           5.46          0.45   46.73
HEALTHCARE           4.76          0.07   29.12
AUTO                 4.27        -16.83   68.20
AC                   3.55         -0.63   34.95
INSURANCE            3.33         -4.67   37.91
DURABLES             3.21         -1.88   39.12
FOOTWEAR             2.38        -27.47   69.58
POWER                2.28         13.44   24.06
METALS               2.17         -4.84   35.92
CERAMICS             1.75        -28.42   71.68
STEEL                1.70          2.30   70.52
TRAVEL               1.63        -43.87  106.13
CHEMICALS            1.51        -14.71  102.43
ENGINEERING          1.21        -17.54   65.83
JEWELLERY            1.14        -85.98  133.71
CEMENT               1.07        -30.09   97.02
MINING               1.05         -5.65   33.06
ENTERTAINMENT        0.90        -33.52  100.65
PHARMA               0.73         -1.16   21.46
TEXTILES             0.57        -19.98   54.19
HOTELS               0.49        -72.62  133.51

In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      2672715.0     26
XR        1106938.0     13
AR        1041138.0     10
X40        721127.0     10
X40N       557043.0     11
XY25       472273.0      8
OX40N      319003.0     11
SR         183576.0      2
MH          84344.0      2
X200        75936.0      1
X5K         74221.0      2

In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2004966.0     20
M-SC        1786700.0     21
H-LC         649363.0     12
X-LC         576939.0      8
H-MC         536167.0      8
X-MC         418986.0      6
M-MC         418910.0      3
X-SC         300221.0      5
M-LC         275878.0      4
L-SC         232154.0      4
L-MC          53439.0      1
L-LC          47082.0      1

In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      905786.0      6
M-SC       XY24      815845.0      7
H-SC       AR        522955.0      3
M-SC       XR        338192.0      4
H-MC       XY24      303534.0      3
M-MC       XY24      302781.0      2
X-LC       X40       298157.0      3
H-SC       XR        221274.0      3
           X40N      210156.0      4
H-LC       X40N      208377.0      4
M-SC       AR        188880.0      2
           OX40N     168128.0      5
H-LC       X40       162998.0      3
M-LC       XY24      160004.0      2
X-MC       X40       158473.0      2
L-SC       XR        152464.0      2
X-SC       XR        123847.0      1
X-LC       AR        120643.0      1
M-SC       XY25      120091.0      1
M-MC       XR        116129.0      1
X-MC       XY25      109474.0      1
M-LC       XR        101593.0      1
X-SC       SR         99705.0      1
X-LC       XY25       98204.0      3
H-SC       OX40N      91690.0      3
H-LC       AR         84424.0      1
M-SC       SR         83871.0      1
H-MC       XY25       83141.0      1
X-MC       XY24       79698.0      1
H-LC       X200       75936.0      1
           X5K        74221.0      2
M-SC       X40        71693.0      1
H-MC       AR         63786.0      1
X-LC       X40N       59935.0      1
L-SC       AR         58558.0      1
X-MC       X40N       53914.0      1
L-MC       XR         53439.0      1
H-SC       MH         53105.0      1
L-LC       XY25       47082.0      1
H-LC       XY24       43407.0      1
X-SC       OX40N      41758.0      2
           XY24       34911.0      1
H-MC       MH         31239.0      1
           X40        29806.0      1
           X40N       24661.0      1
L-SC       XY24       21132.0      1
X-MC       OX40N      17427.0      1
M-LC       XY25       14281.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
